# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [1]:
import sys
sys.path.append('../')
from tokenizer import tokenizer
from vectorizer import vectorizer

import numpy as np
import pandas as pd
import sklearn as sk

[nltk_data] Downloading package stopwords to /home/assil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [3]:
# FOR TESTING : only select first 20000 samples
# X, y = X[:20000], y[:20000]

In [4]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
#tokenized_documents = tokenizer(X)

In [6]:
#X, vect = vectorizer(tokenized_documents)

In [7]:
batch_size = 10000

print(f"Processing data of length {len(X)} in batches of size {batch_size}")
X_vect_batches = []
for start in range(0, len(X), batch_size):
    end = start + batch_size
    print(f"Processing batch from index {start} to {end}")
    batch_X = X[start:end].copy()
    tokenized_documents = tokenizer(batch_X)
    X_vect, _ = vectorizer(tokenized_documents)
    # Store the processed batch
    X_vect_batches.append(X_vect)

X = np.vstack(X_vect_batches)

Processing data of length 568454 in batches of size 10000
Processing batch from index 0 to 10000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 10000 to 20000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 20000 to 30000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 30000 to 40000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 40000 to 50000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 50000 to 60000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 60000 to 70000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 70000 to 80000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 80000 to 90000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 90000 to 100000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 100000 to 110000
Number of tokens:  0

In [8]:
# # most frequent words
# print("Top 10 most frequent words in the dataset")
# print(vect.get_feature_names_out()[:10])

# # least frequent words
# print("Top 10 least frequent words in the dataset")
# print(vect.get_feature_names_out()[-10:])

In [9]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(454763, 300) (113691, 300) (454763,) (113691,)
[0.69675133 0.46625919 0.19659357 0.54161421 0.49475396 0.68864423
 0.77237797 0.61296189 0.64803631 0.67720143 0.53871329 0.43821322
 0.57999926 0.60784278 0.64792152 0.49237756 0.62410983 0.48551687
 0.66650609 0.6353156  0.28711372 0.57419198 0.26327911 0.58957649
 0.58278467 0.46489851 0.33019595 0.34122713 0.50042139 0.47023063
 0.23547583 0.40088003 0.36770556 0.21215492 0.44290871 0.29329761
 0.39150614 0.41075528 0.67269826 0.2800273  0.59147843 0.74466023
 0.64935439 0.45447142 0.53504712 0.59418841 0.35070694 0.20508571
 0.42417751 0.48023969 0.40409482 0.63481863 0.44575853 0.45432337
 0.44045728 0.50378316 0.26800294 0.76815876 0.40490527 0.42683939
 0.6215097  0.43453887 0.51992281 0.53506951 0.58080606 0.52360652
 0.54309204 0.47171951 0.43835253 0.65036521 0.4512514  0.70861387
 0.63950446 0.58614768 0.42354527 0.74754527 0.66673732 0.70711648
 0.60580582 0.34354116 0.45962957 0.76664349 0.34378702 0.61320123
 0.4956486  0.

# RNN
### Model starts here

In [11]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.layers import LeakyReLU
from sklearn import metrics
import datetime
import os

2024-05-18 20:56:20.608608: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 20:56:22.419326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True),
    ModelCheckpoint('model_best.keras', save_best_only=True, monitor='val_loss', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=0.00001)
]

In [13]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

def batch_generator(X, y, batch_size, num_classes):
    num_samples = X.shape[0]
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end]
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield X_batch, y_batch

# Setup the model
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(64))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32, activation='tanh'))
model.add(BatchNormalization())
model.add(Dense(16, activation='tanh'))
model.add(BatchNormalization())
model.add(Dense(np.max(y_train) + 1, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])

# Define the generator
train_generator = batch_generator(X_train, y_train, 64, np.max(y_train) + 1)

# Train the model with gradient clipping
model.fit(train_generator, steps_per_epoch=len(X_train) // 64, epochs=5, 
          callbacks=[#tf.keras.callbacks.ModelCheckpoint('model.h5', save_best_only=True),
                     tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
          verbose=1)

2024-05-18 20:56:23.909142: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-18 20:56:24.412614: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-18 20:56:24.412875: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/5


2024-05-18 20:56:29.656199: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


7105/7105 ━━━━━━━━━━━━━━━━━━━━ 366s 51ms/step - accuracy: 0.5174 - loss: 1.4524
Epoch 2/5
   3/7105 ━━━━━━━━━━━━━━━━━━━━ 6:39 56ms/step - accuracy: 0.6802 - loss: 1.0676

/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


7105/7105 ━━━━━━━━━━━━━━━━━━━━ 371s 52ms/step - accuracy: 0.6380 - loss: 1.1123
Epoch 3/5
7105/7105 ━━━━━━━━━━━━━━━━━━━━ 373s 52ms/step - accuracy: 0.6380 - loss: 1.0995
Epoch 4/5
7105/7105 ━━━━━━━━━━━━━━━━━━━━ 373s 52ms/step - accuracy: 0.6380 - loss: 1.0840
Epoch 5/5
7105/7105 ━━━━━━━━━━━━━━━━━━━━ 374s 53ms/step - accuracy: 0.6421 - loss: 1.0422


In [15]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1)

# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
print("Confusion Matrix:")
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

print("\nClassification Report:")
print(metrics.classification_report(y_test, predictions))

3553/3553 ━━━━━━━━━━━━━━━━━━━━ 53s 15ms/step
Confusion Matrix:
[[10276     0     0     0    50]
 [ 5816     0     0     0    39]
 [ 8372     0     0     0   113]
 [15333     0     0     0   790]
 [65210     0     0     0  7692]]

Classification Report:
              precision    recall  f1-score   support

           1       0.10      1.00      0.18     10326
           2       0.00      0.00      0.00      5855
           3       0.00      0.00      0.00      8485
           4       0.00      0.00      0.00     16123
           5       0.89      0.11      0.19     72902

    accuracy                           0.16    113691
   macro avg       0.20      0.22      0.07    113691
weighted avg       0.58      0.16      0.14    113691



/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [16]:
# # Save the model with current date and time in model folder

# Create a folder named _models in the current directory
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/config3_rnn_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')